# CSC_52081_EP - Lab 02

### Main Objectives 

As per last week, we study an environment in the context of *perception* (observation), *knowledge* (representation), *reasoning* (inference), and *acting* (decision-making). However, today we will additionally study *learning*, and bear in mind aspects of *scalability*; notably the fact that it is usually unreasonable to fully explore and evaluation all possible decisions; and it is typically unreasonable also to assume that we have full and accurate specifification of the environment dynamics (characteristics of the scenaario we are working on). 

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

## Inference as Search 

For inference in the context of multi-output and structured prediction, there is often an intractable number of possibilities; too many to enumerate in a brute force fashion. This is often the case in multi-label classification, and it can be the case in scenarios such as we looked at in Lab 01, where we have a probability tree. In that case, the probability tree was easy to enumerate, but this time we have modified the `Environment` class in `environment.py` to produce a more dense tree (namely, by allowing false-positive sound emissions); and producing longer trajectories. 

In [ ]:
from environment import Environment

# Instantiate the environment
G = np.array([[1,3,0,2,4,1],
              [2,1,0,3,0,3],
              [4,0,3,0,2,0],
              [3,1,2,3,0,4],
              [2,0,0,0,1,1]])
# The fps flag allows for false positives (make sure you have the recent version of envioronment.py)
env = Environment(G,fps=True)
# Generate a single step, it should be possible to see a false positive appear
s, o = env.step()
ooo = np.array([o])
sss = np.array([s]).reshape(1,-1)
fig, ax = env.render(sss, ooo)

So, now let's take a fresh look at Lab 01, specifically the function `P_traj`. Go back to that function and make use of the `M` parameter to allow for abritrarily efficient inference; in particular via **Monte Carlo Search**. Essentially, you should sample `M` trajectories as an approximation:  
$$
    \{s^{(m)}_1,\ldots,s^{(m)}_T, p^{(m)}\}_{m=1}^M \approx P(S_1,\ldots,S_T \mid \vec{x}_1,\ldots,\vec{x}_T)
$$
You can use what is known as **ancestral sampling**, since you have access to the environment, sample $s_1 \sim P(s_1 | \vec{x}_1)$, then $s_2 \sim P(s_2 | s_1)$ and so on (similarly to how you probably did for the brute force solution, but only selecting one of the possibilities. 

Again, this is not the same as simply using `gen_traj`, because we are conditioning on the observation $\vec{x}_1,\ldots,\vec{x}_T$. 

#### Task 

Reimplement `P_traj` where it should use your solution from last week when `M<0` and take `M` samples when `M>0`. 

In [ ]:
# This code is to extract your solution from Lab 01, replace 'Lab_01_Solution.ipynb' as appropriate
from extracter import extract_tagged_cells
extract_tagged_cells("Lab_01_Solution.ipynb","agent.py",tags=["import numpy as np", "def gen_traj", "class Agent"])

In [ ]:
from agent import Agent, gen_traj

In [ ]:
# Generate a trajectory
ooo, sss = gen_traj(env,10)

# Instantiate your agent
agent = Agent(env)

# Use your new implementation, by specifying M>0
P_joint = agent.P_traj(ooo,M=50)

# Create fig
plt.figure()
plt.bar(list(P_joint.keys()), list(P_joint.values()))
plt.xlabel(r"$\mathbf{s}$")
plt.ylabel(r"$p(\mathbf{s}|\mathbf{o})$")
plt.xticks(rotation=45, ha='right')

# Show the plot
plt.tight_layout()
plt.show()

Discussion points: What happens if we only made a longer trajectory, but it was still sparse in probabilities? Would 'ancestral sampling' still be appropriate? 


In [ ]:
# For the remainder, let's use the first version of the enviroment
env = Environment(G)

## Machine Learning for Autonomous Agents

Until now we supposed full unfettered access to the environment. But now suppose that we do *not* have access to the environment definition. Rather, we need to learn it. Suppose that we *do* have access to trajectories from the enviroment, generated by some agent interacting with it,  
$$
    \mathbf{o}_1,\ldots,\mathbf{o}_T \sim p
$$
$$
    \mathbf{a} \sim \pi(\mathbf{o}_1,\ldots,\mathbf{o}_T)
$$
where (in the following) $\mathbf{a}$ is a one-hot-encoded action, relating to the number of possible actions. 

In [ ]:
# We will import pytorch
import torch
import torch.nn as nn

#### Generating data 

Let's generate data, as 
$$
    D = \{(\mathbf{o}_{1:T}, \mathbf{a}_i\}
$$
where $\mathbf{a}$ is one-hot-encoded.  

So dataset `D` will contain $n$ tuples, each containing $T$ observations (dimensionality $2$) and $T$ actions (dimensionality `n_states`). 

This is as if recording an expert agent performing in the environment. Note that we could also include directly the action taken by an agent, rather assuming the ground truth. 

In [ ]:
# Quickly visualise
T=10
ooo, sss = gen_traj(env,T)
fig, ax = env.render(sss, ooo, title=r"$\{s^{(i)}_1,\ldots,s^{(i)}_T\}_{i=1}^n$")
plt.show()

In [ ]:
# Build a dataset
D = []
n_states = env.n_states
trajs = []
n = 1000
T = 6

for i in range(n):
    ooo, sss = gen_traj(env,T)
    trajs.append(sss)
    ooo = torch.tensor(ooo, dtype = torch.float)
    sss = torch.tensor(sss)
    # We also have the (more realistic) option to use the expert agent's action here
    one_hot = torch.nn.functional.one_hot(sss, num_classes=n_states)
    D.append((ooo, one_hot.float()))

#### Transformers

We will be using transformers, which are excellent for sequential modelling. Have a look at the following `class`, which we will use in the following. 

In [ ]:
class TinyTransformerDecoder(nn.Module):
    def __init__(self, input_dim, output_dim, embedding_dim=16, num_layers=1, num_heads=1, max_seq_len=500):
        super(TinyTransformerDecoder, self).__init__()
        
        self.input_projection = nn.Linear(input_dim, embedding_dim)
        
        # Positional encoding
        self.positional_encoding = self._generate_positional_encoding(max_seq_len, embedding_dim)
        
        # Use TransformerEncoder with batch_first=True
        encoder_layer = nn.TransformerEncoderLayer(
            d_model=embedding_dim,
            nhead=num_heads,
            dim_feedforward=embedding_dim * 2,
            dropout=0.,  # small model/dataset
            batch_first=True  # This is the key change
        )
        self.transformer = nn.TransformerEncoder(encoder_layer, num_layers=num_layers)
        
        self.output_projection = nn.Linear(embedding_dim, output_dim)
        
    def _generate_positional_encoding(self, max_seq_len, embedding_dim):
        position = torch.arange(max_seq_len).unsqueeze(1)
        div_term = torch.exp(torch.arange(0, embedding_dim, 2) * (-np.log(10000.0) / embedding_dim))
        pos_encoding = torch.zeros(max_seq_len, embedding_dim)
        pos_encoding[:, 0::2] = torch.sin(position * div_term)
        pos_encoding[:, 1::2] = torch.cos(position * div_term)
        return pos_encoding  # Shape: (max_seq_len, embedding_dim)
        
    def forward(self, x):
        # x shape: (batch_size, seq_len, input_dim)
        x_emb = self.input_projection(x)
        
        # Add positional encoding
        pos_encoding = self.positional_encoding[:x.size(1), :].to(x.device)
        x_emb = x_emb + pos_encoding

        # Generate causal mask
        causal_mask = nn.Transformer.generate_square_subsequent_mask(x.size(1))
        
        # Use mask in the encoder
        x_transformed = self.transformer(x_emb, mask=causal_mask)
        
        output = self.output_projection(x_transformed)
        return output

## Imitation Learning 

Imitation learning is an alternative to reinforcement learning, for producing an autonomous agent. Rather than learn via reinforcement, we can learn to imitate an expert agent (possibly, a human) that has already solved a given task. 

In this case, we would suppose that the agent which has generated the trajectories in data set `D` is such an expert. 

Specifically, in imitation learning (and in this example), we seek
$$
    \mathbf{a}_t = \pi(\mathbf{o}_1, \ldots, \mathbf{o}_t)
$$ 
where $\pi$ the policy of the agent, that maps inputs to an action. 

In the case of the Environment studied in Lab 01, is it clear why we should consider the full history of observations? 

In [ ]:
input_dim = 2
output_dim = n_states
embedding_dim = 16
num_layers = 1
num_heads = 1
seq_length = 6
batch_size = 16

#### Task 

Use the `TinyTransformerDecoder` defined above to complete the Imitation Learning task in the cell below.

**Important** Do not remove the first comment from the cell `# TASK 2 IMITATION LEARNING` as it will help us locate your implementation. 

**Important** Your configuration should complete in less than 5 minutes on a personal laptop machine (noting that 2 minutes should be more than enough)

In [ ]:
# TASK 2 IMITATION LEARNING

model_il = TinyTransformerDecoder(input_dim, output_dim, embedding_dim=embedding_dim, num_layers=num_layers, num_heads=num_heads)
dataloader = torch.utils.data.DataLoader(D, batch_size=batch_size, shuffle=True)

# TODO  

### Evaluation

How well is our model (agent) performing (the one learned via imitation learning)?

In the following, we deploy the agent and test it, in a similar way to Lab 01 (on the same environment). 

First, we wrap the model inference in the `act` function of an Agent class (namely, `DTAgent` below), so we can compare. 

#### Task

Complete the `act` function in the `DTAgent` class below to return an integer indicating the action, as per the description of the scenario given already in Lab 01. 

Hint: to get probabilities, use `torch.softmax(outputs, axis = 2)`

In [ ]:

class DTAgent:

    def act(self,ooo):
        '''
        Decide on the best action to take, under the provided observation. 

        Parameters
        ----------

        ooo : array_like(int,ndim=2)
            t observations (of 2 bits each)

        Returns
        -------

        a : int
            the chosen action a, it should be one of the n_states
        '''

        # Predict
        input_tensor = torch.tensor(ooo, dtype=torch.float32).unsqueeze(0)  
        # TODO 

In the following, we perform the evaluation. 

In [ ]:
# Instantiate the agent
dt_agent = DTAgent()
n_test = 1000
T_test = 6
score = 0

# Generate some trajectories from the environment 
n_test=1000
for i in range(n_test): 
    oooo, ssss = gen_traj(env,T_test)
    oooo = torch.tensor(oooo, dtype = torch.float32)
    a_t = dt_agent.act(oooo)
    score += env.rwd(a_t,ssss[-1])
score = score/n_test
print(score)

## Model Learning

Imitation Learning in the real world can be a bit risky. If we had a *model* of the environment, we could 'practice'. Namely, we seek
$$
    \mathbf{x}_{t+1} \sim P(\mathbf{x}_{t+1} | \mathbf{x}_1, \ldots, \mathbf{x}_t)
$$ 

With this we could generate trajectories, with which to safely train an agent in simulation. 

In [ ]:
input_dim = 2
output_dim = 2
embedding_dim = 16
num_layers = 1
num_heads = 1
seq_length = 6
batch_size = 16

#### Task 

Use the `TinyTransformerDecoder` to complete the Model Learning task in the cell below. Again, do not replace or remove the `TASK` tag.

Then, complete the `sample_next_obs` function of the `DTEnvironment` class to sample 

**Important** Do not remove the first comment from the cell `# TASK 3 MODEL LEARNING` as it will help us locate your implementation. 

**Important** Your configuration should complete in less than 5 minutes on a personal laptop machine (noting that 2 minutes should be more than enough)

In [ ]:
# TASK 3 MODEL LEARNING
model_ml = TinyTransformerDecoder(input_dim, output_dim, embedding_dim=embedding_dim, num_layers=num_layers, num_heads=num_heads)
dataloader = torch.utils.data.DataLoader(D, batch_size=batch_size, shuffle=True)

# TODO  

In [ ]:
class DTEnvironment:

    def sample_next_obs(self,ooo):
        '''
        Provide a next observation, to follow the provided observation. 

        Parameters
        ----------

        ooo : array_like(ndim=2)
            t observations (of 2 bits each)

        Returns
        -------

        o : array_like(ndim=1)
            the next observation as an array of 2 ints
        '''

        input_tensor = torch.tensor(ooo, dtype=torch.float32)

        model_ml.eval()
        with torch.no_grad():  # Disable gradient comp for inference
            obs = model_ml(input_tensor.unsqueeze(0))
        obs = torch.sigmoid(obs)

        # Extract the next observation
        final_obs = obs[:, -1, :]
        
        return torch.bernoulli(final_obs).int().numpy()

Once we have a model of the environment, we can simulate new trajectories, as exemplified as follows. 

In [ ]:
dt_env = DTEnvironment()
# Generate some trajectories from the environment 
T=10
oooo = np.zeros((T,2))
for t in range(1,T):
    out = dt_env.sample_next_obs(oooo[0:t,:])
    oooo[t,:] = dt_env.sample_next_obs(oooo[0:t,:])

print(oooo)

You can use your `p_traj` function to check the 'legitimacy' of these trajectories (they should have probability greater than 0). 

Of course, in a real-world environment, validating the trajectories is more subjective, or requires human expertise. 
# 

In [ ]:
agent = Agent(env)
P_joint = agent.P_traj(oooo.astype(int))

# Check should sum to 1 (or close enough)
probs = list(P_joint.values())
assert abs(sum(probs) - 1) <= 0.05

# Extract possible paths
paths = [np.fromstring(k, sep=' ') for k in P_joint.keys()] 

# Take some samples
sample_indices = np.random.choice(len(probs), size=10, p=probs)
trajs = [paths[i].astype(int) for i in sample_indices]

# Visualise 
fig, ax = env.render(trajs[0], oooo, title=r"$s_1,\ldots,s_T \sim \hat P_\theta(\cdot \mid o_1,\ldots,o_T)$")
plt.show()

## A Shared Representation

Notice that both the previous tasks (imitation learning and model learning) have something in common; namely the input. It suggests it may be more efficient to share a representation in a deep architecture, rather than reimplementing both as separate networks. Such architectures will also be very useful later for reinforcement learning. 

Have a look at the architecture defined in the following. 

In [ ]:
class TinyTransformerDecoderDoubleHead(nn.Module):
    def __init__(self, input_dim, act_dim, obs_dim, embedding_dim=16, num_layers=1, num_heads=1, max_seq_len=500):
        super(TinyTransformerDecoderDoubleHead, self).__init__()
        
        self.input_projection = nn.Linear(input_dim, embedding_dim)
        
        # Positional encoding
        self.positional_encoding = self._generate_positional_encoding(max_seq_len, embedding_dim)
        
        # Use TransformerEncoder with batch_first=True
        encoder_layer = nn.TransformerEncoderLayer(
            d_model=embedding_dim,
            nhead=num_heads,
            dim_feedforward=embedding_dim * 2,
            dropout=0.,  # small model/dataset
            batch_first=True  # This is the key change
        )
        self.transformer = nn.TransformerEncoder(encoder_layer, num_layers=num_layers)
        
        self.act_projection = nn.Linear(embedding_dim, act_dim)
        self.obs_projection = nn.Linear(embedding_dim, obs_dim)
        
    def _generate_positional_encoding(self, max_seq_len, embedding_dim):
        position = torch.arange(max_seq_len).unsqueeze(1)
        div_term = torch.exp(torch.arange(0, embedding_dim, 2) * (-np.log(10000.0) / embedding_dim))
        pos_encoding = torch.zeros(max_seq_len, embedding_dim)
        pos_encoding[:, 0::2] = torch.sin(position * div_term)
        pos_encoding[:, 1::2] = torch.cos(position * div_term)
        return pos_encoding  # Shape: (max_seq_len, embedding_dim)
        
    def forward(self, x):
        # x shape: (batch_size, seq_len, input_dim)
        x_emb = self.input_projection(x)
        
        # Add positional encoding
        pos_encoding = self.positional_encoding[:x.size(1), :].to(x.device)
        x_emb = x_emb + pos_encoding

        # Generate causal mask
        causal_mask = nn.Transformer.generate_square_subsequent_mask(x.size(1))
        
        # Use mask in the encoder
        x_transformed = self.transformer(x_emb, mask=causal_mask)
        
        act = self.act_projection(x_transformed)
        obs = self.obs_projection(x_transformed)
        return act, obs

In [ ]:
input_dim = 2
act_dim = 30
obs_dim = 2
embedding_dim = 16
num_layers = 1
num_heads = 1
seq_length = 6
batch_size = 16

#### Task 

Implement the Architecture in PyTorch using a double head (and shared inner representations). 

In [ ]:
# TASK 4 SHARED REPRESENTATION 
model_shared = TinyTransformerDecoderDoubleHead(input_dim, act_dim, obs_dim, embedding_dim=embedding_dim, num_layers=num_layers, num_heads=num_heads)
dataloader = torch.utils.data.DataLoader(D, batch_size=batch_size, shuffle=True)

# TODO  